# Ein Hybridmodell mit LSTM-AE, DBSCAN und IF

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import logging
from tensorflow.keras.layers import TimeDistributed, Dense, LSTM, Dropout, LayerNormalization, Input, Add, Bidirectional, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
import sys
from keras.layers import LSTM, Dropout, LayerNormalization, Add, TimeDistributed, Dense
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from tensorflow.keras import backend as K

ModuleNotFoundError: No module named 'numpy'

## 1. Vorbereitung

#### Cuda auslassen, weil DGX verwendet wird

#### Daten laden

In [ ]:
data_train = pd.read_json("train_logs.json", lines=False)
data_test = pd.read_json("test_logs.json", lines=False)
data_val = pd.read_json("val_logs.json", lines=False)
print(data_test.head())

#### NaN-Werte bereinigen

In [ ]:
X_train_full = pd.DataFrame(data_train)
X_test_full = pd.DataFrame(data_test)
X_val_full = pd.DataFrame(data_val)

y_test_full = data_test["label"]
y_val_full = data_val["label"]

#### Logging

In [ ]:
logfile = open("output_log.txt", "w")
logging.basicConfig(stream=logfile, level=logging.INFO)

sys.stdout = logfile
sys.stderr = logfile

logging.info("Modelle wurden gestartet.")

#### Automatisch alles in Numerische oder Kategorische Daten einteilen

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def is_missing(val):
    return val is None or (isinstance(val, float) and np.isnan(val))

def auto_encode_features(logs, one_hot_numeric=False, 
                         label_encoders=None, onehot_encoders=None, fit=True):
    if logs is None or len(logs) == 0:
        return [], {}, {}

    if hasattr(logs, "to_dict"):
        logs = logs.to_dict(orient='records')

    if label_encoders is None:
        label_encoders = {}
    if onehot_encoders is None:
        onehot_encoders = {}

    sample = logs[0]
    all_keys = sample.keys()
    encoded_logs = []

    for log in logs:
        encoded = {}
        for key in all_keys:
            val = log.get(key)

            # Fehlt etwas? → Speziell ersetzen
            if is_missing(val):
                val = "__MISSING__"

            # Numerisch?
            if isinstance(val, (int, float)) and not isinstance(val, bool):
                if val == "__MISSING__":
                    encoded[key] = -9999  # spezieller Platzhalter für fehlende Zahl
                elif one_hot_numeric:
                    if key not in onehot_encoders and fit:
                        values = np.array([[l.get(key) if not is_missing(l.get(key)) else -9999]
                                           for l in logs])
                        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
                        encoder.fit(values)
                        onehot_encoders[key] = encoder

                    if key in onehot_encoders:
                        enc = onehot_encoders[key].transform([[val]])[0]
                        for i, v in enumerate(enc):
                            encoded[f"{key}_{i}"] = v
                else:
                    encoded[key] = val

            # String (kategorisch)
            elif isinstance(val, str):
                if key not in label_encoders and fit:
                    values = list(set(l.get(key) if not is_missing(l.get(key)) else "__MISSING__"
                                      for l in logs))
                    le = LabelEncoder()
                    le.fit(values)
                    label_encoders[key] = le

                if key in label_encoders:
                    le = label_encoders[key]
                    if val in le.classes_:
                        encoded[key] = le.transform([val])[0]
                    else:
                        encoded[key] = -1  # unbekannte Kategorie

            # Listen
            elif isinstance(val, list):
                for i, item in enumerate(val):
                    if is_missing(item):
                        item = "__MISSING__"

                    label = f"{key}_{i}"
                    if label not in label_encoders and fit:
                        values = list(set(itm if not is_missing(itm) else "__MISSING__"
                                          for l in logs for itm in l.get(key, [])))
                        le = LabelEncoder()
                        le.fit(values)
                        label_encoders[label] = le

                    if label in label_encoders:
                        le = label_encoders[label]
                        if item in le.classes_:
                            encoded[label] = le.transform([item])[0]
                        else:
                            encoded[label] = -1

        encoded_logs.append(encoded)

    return encoded_logs, label_encoders, onehot_encoders

#### Werte aus vorheriger Funktion vereinigen

In [ ]:
def dicts_to_feature_matrix(encoded_logs):
    feature_names = sorted({key for d in encoded_logs for key in d.keys()})

    X = np.zeros((len(encoded_logs), len(feature_names)), dtype=np.float32)

    for i, d in enumerate(encoded_logs):
        for j, feat in enumerate(feature_names):
            if feat in d:
                X[i, j] = d[feat]

    return X, feature_names

In [ ]:
logs_train = X_train_full.to_dict(orient='records')
logs_test = X_test_full.to_dict(orient='records')
logs_val = X_val_full.to_dict(orient='records')

encoded_logs, label_encoders, onehot_encoders = auto_encode_features(
    logs_train, one_hot_numeric=True, fit=True
)
X_train, feature_names = dicts_to_feature_matrix(encoded_logs)

encoded_test_logs, _, _ = auto_encode_features(
    logs_test, one_hot_numeric=True,
    label_encoders=label_encoders, onehot_encoders=onehot_encoders, fit=False
)
X_test, _ = dicts_to_feature_matrix(encoded_test_logs)

encoded_val_logs, _, _ = auto_encode_features(
    logs_val, one_hot_numeric=True,
    label_encoders=label_encoders, onehot_encoders=onehot_encoders, fit=False
)
X_val, _ = dicts_to_feature_matrix(encoded_val_logs)

#### Skalierung

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

#### PCA zur Dimensionsreduktion

### Parameter

In [ ]:
seq_length = 15
batch_size = 128

encoder_layers = [1024, 512, 256]
decoder_layers = [256, 512, 1024]

dropout_rate = 0.2
learning_rate = 0.0001

### Sequenzen

In [ ]:
class SequenceToSequenceGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, seq_length, batch_size, pad_last=True):
        self.data = data
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.pad_last = pad_last

        self.indices = np.arange(len(data) - seq_length)
        
        # Wenn Padding aktiv ist und etwas übrig bleibt
        if pad_last and (len(data) % seq_length != 0):
            self.include_last = True
        else:
            self.include_last = False

    def __len__(self):
        base = (len(self.indices) + self.batch_size - 1) // self.batch_size
        return base + (1 if self.include_last else 0)

    def __getitem__(self, idx):
        if idx < len(self) - 1 or not self.include_last:
            batch_idx = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
            X_batch = np.array([self.data[i:i + self.seq_length] for i in batch_idx])
        else:
            # letzte Sequenz mit Padding
            last_seq = self.data[-self.seq_length:]
            if len(last_seq) < self.seq_length:
                padding_len = self.seq_length - len(last_seq)
                padding = np.zeros((padding_len, self.data.shape[1]))
                last_seq = np.vstack((last_seq, padding))
            X_batch = np.expand_dims(last_seq, axis=0)

        return X_batch, X_batch


train_gen = SequenceToSequenceGenerator(X_train_scaled, seq_length, batch_size)
val_gen = SequenceToSequenceGenerator(X_val_scaled, seq_length, batch_size)
test_gen = SequenceToSequenceGenerator(X_test_scaled, seq_length, batch_size)

## 2. LSTM-AE implementieren

In [ ]:
#  LSTM-Block
def residual_lstm_block(inputs, units, dropout_rate=dropout_rate):
    shortcut = inputs
    x = LSTM(units, return_sequences=True)(inputs)
    x = Dropout(dropout_rate)(x)
    x = LayerNormalization()(x)

    # Falls Dimension nicht passt, anpassen
    if K.int_shape(shortcut)[-1] != units:
        shortcut = TimeDistributed(Dense(units))(shortcut)

    x = Add()([x, shortcut])
    return x

n_features = X_train_scaled.shape[1]

inputs = Input(shape=(seq_length, n_features))
x = inputs

# Encoder mit Residual-Blöcken
for units in encoder_layers:
    x = residual_lstm_block(x, units, dropout_rate)

# Bottleneck
encoded = Bidirectional(LSTM(encoder_layers[-1]))(x)

# Decoder
decoded = RepeatVector(seq_length)(encoded)
x = decoded

for units in decoder_layers:
    x = residual_lstm_block(x, units, dropout_rate) 

outputs = TimeDistributed(Dense(n_features, activation='linear'))(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate), loss='mse')
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[early_stop])

### Umwandlung der Daten für nächste Modelle: Rekonstruktionsfehler berechnen

In [ ]:
def get_reconstruction_errors(gen, model):
    errors = []
    for i in range(len(gen)):
        X_batch, _ = gen[i]
        pred = model.predict(X_batch, verbose=0)
        batch_errors = np.mean(np.square(X_batch - pred), axis=(1, 2))
        errors.extend(batch_errors)
    return np.array(errors)

train_errors = get_reconstruction_errors(train_gen, model)
val_errors = get_reconstruction_errors(val_gen, model)
test_errors = get_reconstruction_errors(test_gen, model)

reconstruction_errors = get_reconstruction_errors(test_gen, model)
threshold = np.percentile(train_errors, 95)

In [ ]:
y_test_seq = np.array([y_test_full[i + seq_length - 1] for i in range(len(reconstruction_errors))])
true_labels = y_test_seq.astype(int)

fpr, tpr, thresholds = roc_curve(true_labels, reconstruction_errors)
optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]
y_pred = (reconstruction_errors > optimal_threshold).astype(int)

precision, recall, f1, _ = precision_recall_fscore_support(true_labels, y_pred, average='binary')
roc_auc = roc_auc_score(true_labels, reconstruction_errors)
logging.info(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1-Score: {f1:.3f}, ROC-AUC: {roc_auc:.3f}")

In [ ]:
scaler_rec = StandardScaler()
scaler_rec.fit(train_errors.reshape(-1, 1))
val_errors_scaled = scaler_rec.transform(val_errors.reshape(-1, 1))
test_errors_scaled = scaler_rec.transform(test_errors.reshape(-1, 1))

# Modell 1

## Isolation Forest

In [ ]:
iforest = IsolationForest()
iforest.fit(train_errors.reshape(-1, 1))

test_preds_if = iforest.predict(test_errors.reshape(-1, 1))
val_anomaly_if = (test_preds_if == -1).astype(int)

y_test_seq_if = np.array([y_test_full[i + seq_length- 1] for i in range(len(val_anomaly_if))])
true_labels_if = y_test_seq_if.astype(int)

report_iforest_if = classification_report(true_labels_if, val_anomaly_if)
logging.info("Isolation Forest - Test Report:\n" + report_iforest_if)

### IF MCC

In [ ]:
mcc_if = matthews_corrcoef(true_labels_if, val_anomaly_if)
logging.info("Matthews Correlation Coefficient: %f", mcc_if)

### IF Balanced Accuracy

In [ ]:
balanced_acc = balanced_accuracy_score(true_labels_if, val_anomaly_if)
logging.info("Balanced Accuracy: %f", balanced_acc)

### IF AUC-Kurven

In [ ]:
val_scores = -iforest.decision_function(val_errors.reshape(-1,1))
y_val_seq = np.array([y_val_full[i + seq_length - 1] for i in range(len(val_scores))])
true_labels = y_val_seq.astype(int)

fpr, tpr, _ = roc_curve(true_labels, val_scores)
roc_auc = roc_auc_score(true_labels, val_scores)

precision, recall, _ = precision_recall_curve(true_labels, val_scores)
pr_auc = average_precision_score(true_labels, val_scores)

In [ ]:
plt.figure(figsize=(12, 5))

# ROC Plot
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Validation)")
plt.legend()
plt.tight_layout()
plt.savefig('roc_curve_if.png', dpi=300)
plt.close()

In [ ]:
# PR Plot
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AP = {pr_auc:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Validation)")
plt.legend()
plt.tight_layout()
plt.savefig('pr_curve_if.png', dpi=300)
plt.close()

# Modell 2

## One-Class SVM

In [ ]:
ocsvm = OneClassSVM(nu=0.005, gamma=50)
ocsvm.fit(train_errors.reshape(-1, 1))

test_preds_ocsvm = ocsvm.predict(test_errors.reshape(-1, 1))
test_anomaly_ocsvm = (test_preds_ocsvm == -1).astype(int)

y_test_seq_ocsvm = np.array([y_test_full[i + seq_length - 1] for i in range(len(test_anomaly_ocsvm))])
true_labels_ocsvm = y_test_seq.astype(int)

report_ocsvm = classification_report(true_labels_ocsvm, test_anomaly_ocsvm)
logging.info("One-Class SVM - Test Report:\n" + report_ocsvm)

### OCSVM MCC

In [ ]:
mcc = matthews_corrcoef(true_labels_ocsvm, test_anomaly_ocsvm)
logging.info("Matthews Correlation Coefficient: %f", mcc)

### OCSVM Balanced Accuracy

In [ ]:
balanced_acc = balanced_accuracy_score(true_labels_ocsvm, test_anomaly_ocsvm)
logging.info("Balanced Accuracy: %f", balanced_acc)

### OCSVM AUC-Kurven

In [ ]:
val_scores_ocsvm = -ocsvm.decision_function(val_errors.reshape(-1,1))
y_val_seq_ocsvm = np.array([y_val_full[i + seq_length - 1] for i in range(len(val_scores_ocsvm))])
true_labels_ocsvm = y_val_seq_ocsvm.astype(int)

fpr, tpr, _ = roc_curve(true_labels_ocsvm, val_scores_ocsvm)
roc_auc = roc_auc_score(true_labels_ocsvm, val_scores_ocsvm)

precision, recall, _ = precision_recall_curve(true_labels_ocsvm, val_scores_ocsvm)
pr_auc = average_precision_score(true_labels_ocsvm, val_scores_ocsvm)
# Plotten
plt.figure(figsize=(12, 5))

# ROC OCSVM
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Validation)")
plt.legend()
plt.tight_layout()
plt.savefig('roc_curve_ocsvm.png', dpi=300)
plt.close()

# PR OCSVM
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AP = {pr_auc:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Validation)")
plt.legend()
plt.tight_layout()
plt.savefig('pr_curve_ocsvm.png', dpi=300)
plt.close()

# Modell 3

## DBSCAN

In [ ]:
dbscan = DBSCAN(eps = 0.05, min_samples = 40)

dbscan_labels = dbscan.fit_predict(test_errors.reshape(-1, 1))
dbscan_anomaly = (dbscan_labels == -1).astype(int)

y_test_seq_dbscan = np.array([y_test_full[i + seq_length - 1] for i in range(len(dbscan_anomaly))])
true_labels_dbscan = y_test_seq_dbscan.astype(int)
logging.info("\n" + classification_report(true_labels_dbscan, dbscan_anomaly))

### DBSCAN MCC

In [ ]:
mcc = matthews_corrcoef(true_labels_dbscan, dbscan_anomaly)
logging.info("Matthews Correlation Coefficient: %f", mcc)

### DBSCAN Balanced Accuracy

In [ ]:
balanced_acc = balanced_accuracy_score(true_labels_dbscan, dbscan_anomaly)
logging.info("Balanced Accuracy: %f", balanced_acc)

### DBSCAN AUC-Kurven

In [ ]:
scores = dbscan_anomaly

fpr, tpr, _ = roc_curve(true_labels_dbscan, scores)
roc_auc = roc_auc_score(true_labels_dbscan, scores)

precision, recall, _ = precision_recall_curve(true_labels_dbscan, scores)
pr_auc = average_precision_score(true_labels_dbscan, scores)
plt.figure(figsize=(12, 5))

In [ ]:
# ROC-Kurve
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (DBSCAN)")
plt.legend()
plt.savefig('roc_curve_dbscan.png', dpi=300)
plt.close()

In [ ]:
# PR-Kurve
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AP = {pr_auc:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (DBSCAN)")
plt.legend()
plt.tight_layout()
plt.savefig('pr_curve_dbscan.png', dpi=300)
plt.close()